---
---

# **`CGM - Assignment` : Predicting Height of the Children 👧👦📏** 

- Malnutrition is a global health crisis and is a leading cause of death among children under 5 years. 

- Detecting malnutrition requires `anthropometric measurements` of weight, height, and middle-upper arm circumference. 
    - Anthropometric measurements are a non invasive, inexpensive, and suitable method for `evaluating the nutritional status` in the samples.
    - However, anthropometric techniques are `prone to errors` that could arise, for example, from the inadequate training of personnel.


- Therefore, using a CNN-based deep learning model can certainly help in faster, easier and more precised collection of such measurements.


---
---

## **`Examples of various measurements` are shown below:**
![img_1](E:\STUDY\PROJECTS\Assignment_CGM\Doc_Images\Anthropometric_Measurements.png)


---
---


## **`NOTEBOOK Description` 📗**

- This notebook contains the below tasks:
    - `Exploratory Data Analysis`
        - To find some patterns in the provided data
        - Apply some normalizing techniques so that: 
            - performing edge detection will be easier, and
            - noise will be reduced
    - `Added 34 new features`
        - Given the overall bbox co-ordinates & body part specific co-ordinates:
            - calculated the TOP to BODY PART distance, and
            - calculated the BODY PART to BOTTOM distance

## **`NOTEBOOK Contents` ✍️**

[1. Libraries Import](#Import_Libraries)

[2. Dataset Import](#Dataset_Import)

##### ⭐ **`Adding some new Features`**

[1. How we can leverage the provided BBox co-ordinates❓](#Calculate_Distances)

##### ⭐ **`Extensive Data Analysis - I`**

[1. Let's visualize the Feature-1: NOSE Top dist](#Feature-1:NOSE_top_dist)

[2. Let's visualize the Feature-2: NOSE Bottom dist](#Feature-2:NOSE_bottom_dist)

[3. Let's visualize the Feature-3: LEFT EYE Top dist](#Feature-3:LEFT_EYE_top_dist)

[4. Let's visualize the Feature-4: RIGHT SHOULDER Top dist](#Feature-4:RIGHT_SHOULDER_top_dist)

[5. Let's visualize the Feature-5: RIGHT ELBOW Top dist](#Feature-5:RIGHT_ELBOW_top_dist)

[6. Let's visualize the Feature-6: LEFT WRIST Top dist](#Feature-6:LEFT_WRIST_top_dist)

[7. Let's visualize the Feature-7: RIGHT HIP Top dist](#Feature-7:RIGHT_HIP_top_dist)

[8. Let's visualize the Feature-8: LEFT KNEE Bottom dist](#Feature-8:LEFT_KNEE_bottom_dist)

[9. Let's visualize the Feature-9: RIGHT ANKLE Bottom dist](#Feature-9:RIGHT_ANKLE_bottom_dist)

##### ⭐ **`Extensive Data Analysis - II`**

[1. Lets plot some depth images here..](#Lets_plot_some_depth_images_here..)

[2. 🙋‍♂️ Why in some of the images the boundary pixels of the children are too dark or black❓ And, in others they are just like the inner region❓](#Query)

##### ⭐ **`Extensive Data Analysis - III`**

[1. 🧮 Image Smoothing](#⭐Image_Smoothing)

[2. 🧮 Image Gradients & Edges](#⭐Image_Gradients_&_Edges)

[3. 🧮 Sobel v/s Canny Edge Detection Techniques](#⭐Sobel_v/s_Canny_Edge_Detection_Techniques)


[⭐ SUMMARY](#SUMMARY)

## Import_Libraries

In [1]:
import os
import sys
import math
import scipy as scipy
import numpy as np
from numpy import arange
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import ast
import pathlib

import plotly.express as px
import plotly.graph_objects as go
import hashlib
from IPython.display import display
from PIL import Image
from tqdm import tqdm
tqdm.pandas()

%matplotlib inline

In [2]:
# Setting the display option
pd.set_option('display.max_columns', 50)

# Defining the variables for better plotting 
label_font_dict = {'family':'sans-serif','size':13.5,'color':'brown','style':'italic'}
title_font_dict = {'family':'sans-serif','size':17.5,'color':'Blue','style':'italic'}

## **Dataset_Import**

In [3]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 24.9 MB/s 


In [4]:
## TRAIN set files
!gdown 1kuGJodL48MaAb57j2VSbSX_qXellGrBC
!gdown 1NAwb73kvK_uut71vNIte34tiaDKfSmTZ

Downloading...
From: https://drive.google.com/uc?id=1kuGJodL48MaAb57j2VSbSX_qXellGrBC
To: /content/Dataset.zip
100% 203k/203k [00:00<00:00, 117MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NAwb73kvK_uut71vNIte34tiaDKfSmTZ
To: /content/HP_data_engg.csv
100% 3.55k/3.55k [00:00<00:00, 6.52MB/s]


In [5]:
!unzip Dataset.zip

Archive:  Dataset.zip
  inflating: Dataset/1d5fee00-ada4-11eb-a80b-3f50af21830f.jpg  
  inflating: Dataset/1eddf4c0-ad9f-11eb-a80b-ef4adba049aa.jpg  
  inflating: Dataset/38e29bc0-8fc0-11eb-b5f2-0742da91b282.jpg  
  inflating: Dataset/6cd5d480-8b8b-11eb-b152-f7b115384fa9.jpg  
  inflating: Dataset/76ed6fc0-8e1d-11eb-ad70-7be1a1219834.jpg  
  inflating: Dataset/8d23fa60-9201-11eb-b3e6-cbfb48c6a300.jpg  
  inflating: Dataset/9dc3eba0-8d8d-11eb-8d3f-4776be867c61.jpg  
  inflating: Dataset/9dd020a0-8d8d-11eb-8d3f-bfd9f6bd04d4.jpg  
  inflating: Dataset/d3757800-8dff-11eb-a328-472d4f8db277.jpg  


In [6]:
# Storing the location of the 'height_and_pose' file
hp_df = pd.read_csv(os.path.join(os.getcwd(), "HP_data_engg.csv"), index_col="Unnamed: 0")
hp_df['Img_name'] = hp_df['Img_name'].apply(lambda name: os.path.join(os.path.join(os.getcwd(), "Dataset"), name+".jpg"))
hp_df

,Img_name,nose_top_dist,nose_bottom_dist,left_eye_top_dist,left_eye_bottom_dist,right_eye_top_dist,right_eye_bottom_dist,left_ear_top_dist,left_ear_bottom_dist,right_ear_top_dist,right_ear_bottom_dist,left_shoulder_top_dist,left_shoulder_bottom_dist,right_shoulder_top_dist,right_shoulder_bottom_dist,left_elbow_top_dist,left_elbow_bottom_dist,right_elbow_top_dist,right_elbow_bottom_dist,left_wrist_top_dist,left_wrist_bottom_dist,right_wrist_top_dist,right_wrist_bottom_dist,left_hip_top_dist,left_hip_bottom_dist,right_hip_top_dist,right_hip_bottom_dist,left_knee_top_dist,left_knee_bottom_dist,right_knee_top_dist,right_knee_bottom_dist,left_ankle_top_dist,left_ankle_bottom_dist,right_ankle_top_dist,right_ankle_bottom_dist,Height
0,/content/Dataset/1d5fee00-ada4-11eb-a80b-3f50a...,139.691,453.810,151.440,460.771,113.032,476.286,180.259,445.126,101.246,478.053,224.946,388.402,163.677,412.205,291.861,332.618,218.935,368.847,319.802,276.817,235.590,342.005,352.147,229.775,330.627,247.948,457.988,118.041,400.249,210.554,426.089,152.056,458.407,194.857,76.7
1,/content/Dataset/1eddf4c0-ad9f-11eb-a80b-ef4ad...,219.240,376.907,107.062,464.210,208.241,395.238,139.040,432.574,202.198,386.158,170.682,399.706,237.697,349.583,130.723,440.394,287.228,340.067,95.316,481.054,291.823,375.793,306.436,272.108,332.235,238.045,374.111,226.972,393.302,177.855,447.702,165.827,464.532,105.850,66.4
2,/content/Dataset/38e29bc0-8fc0-11eb-b5f2-0742d...,114.684,464.978,121.393,460.750,209.028,415.961,139.339,438.516,210.612,412.764,180.103,396.358,244.709,353.241,117.177,462.027,254.459,392.036,107.047,483.412,254.249,442.271,334.531,250.052,365.415,211.029,409.651,202.716,429.890,146.674,489.473,148.322,489.456,103.502,63.2
3,/content/Dataset/6cd5d480-8b8b-11eb-b152-f7b11...,150.096,505.597,155.437,516.805,127.744,524.638,185.402,504.764,125.420,520.907,242.182,436.003,178.191,464.880,308.353,359.789,245.235,406.229,362.151,296.329,311.701,352.524,369.434,274.015,348.848,301.257,464.828,182.184,445.466,225.565,543.030,128.115,531.762,177.651,87.1
4,/content/Dataset/76ed6fc0-8e1d-11eb-ad70-7be1a...,91.655,500.699,88.932,502.501,89.834,501.079,122.090,470.116,167.362,470.016,160.697,430.627,194.623,439.693,129.537,468.497,212.710,471.281,84.680,507.193,183.968,515.070,308.260,285.770,327.234,268.397,381.836,239.404,422.678,168.536,479.981,165.535,507.097,90.993,59.1
5,/content/Dataset/8d23fa60-9201-11eb-b3e6-cbfb4...,190.423,472.569,179.159,487.789,172.724,488.188,131.322,488.105,136.639,483.716,169.193,435.949,209.694,408.771,239.258,375.096,310.914,304.460,271.681,333.434,289.150,337.040,329.403,282.248,353.680,251.499,447.742,172.448,467.127,140.664,547.518,117.861,567.176,64.311,86.4
6,/content/Dataset/9dc3eba0-8d8d-11eb-8d3f-4776b...,142.212,458.133,136.621,473.452,116.796,480.265,155.741,472.965,103.867,489.563,221.702,394.308,180.226,415.960,300.493,308.431,263.732,342.875,364.402,238.180,337.507,279.494,329.515,265.448,312.568,285.209,422.124,171.750,414.752,195.807,488.571,113.838,492.155,138.798,100.6
7,/content/Dataset/9dd020a0-8d8d-11eb-8d3f-bfd9f...,118.411,437.884,110.964,454.636,96.762,457.973,135.269,443.555,101.644,451.652,199.977,375.700,158.455,396.762,280.915,285.278,247.099,316.309,344.566,216.070,329.922,244.375,325.652,228.244,312.485,246.740,409.378,146.260,405.849,163.278,479.062,91.341,475.298,110.771,106.1
8,/content/Dataset/d3757800-8dff-11eb-a328-472d4...,179.026,501.837,174.395,509.670,176.691,516.647,111.132,530.799,170.003,517.505,173.186,468.816,238.792,433.049,277.820,377.785,333.128,325.585,348.933,315.665,400.402,253.969,354.905,290.880,378.720,261.624,464.386,197.124,471.840,167.764,571.270,159.135,583.359,62.913,99.9


In [7]:
hp_df.shape

(9, 36)

## **`Data_Preparation`**

In [10]:
import random as rn
import datetime
import time
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, Activation, Flatten, Conv1D, Conv2D, Concatenate, Lambda
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import MaxPool1D, MaxPool2D, GlobalAveragePooling1D, GlobalAveragePooling2D, AveragePooling1D, AveragePooling2D, SpatialDropout2D
from keras.regularizers import l1, l2
from keras.preprocessing import image
from keras.callbacks import Callback
from keras.callbacks import TensorBoard
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, roc_auc_score, confusion_matrix
from tensorboard import notebook
from tensorflow.keras.models import model_from_json
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Loading the required libraries
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
import json

ImportError: cannot import name 'mean_absolute_percentage_error'

In [ ]:
!gdown 1YJWChf3MmiiKs86fpXww6_1jnoY68Jlb
!gdown 1sTuiWCHjKQJIjAHigapEFCZ1bKXqzJZw

In [9]:
# loading Models' JSON and creating the Model
with open('resnet_50b.json', 'r') as model_json_file:
    loaded_model_json = model_json_file.read()
    
# Loading the model from json
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("weights-30-193.21.hdf5")

TypeError: ('Keyword argument not understood:', 'keepdims')